In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os 
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader

## Define class CatAndDogDataset

In [56]:
class CatAndDogDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        image_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])

        image = Image.open(image_path)
        label = torch.tensor(int(self.annotations.iloc[index,1]))

        if self.transform:
            image = self.transform(image)

        return (image, label)

## Prepare Dataset

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [58]:
# Hyperparamenter

in_chanel = 3
num_class = 2
learning_rate = 0.001
batch_size = 32
num_epochs = 10

In [59]:
# Load data
dataset = CatAndDogDataset(
    csv_file = "/kaggle/input/annotations/cats_dogs.csv"
    , root_dir = "/kaggle/input/cat-and-dog"
    , transform = transforms.ToTensor()
)

In [60]:
dataset[0]

(tensor([[[0.7922, 0.7961, 0.8118,  ..., 0.9529, 0.9569, 0.9490],
          [0.7922, 0.7961, 0.8118,  ..., 0.9569, 0.9608, 0.9529],
          [0.7961, 0.8000, 0.8118,  ..., 0.9608, 0.9647, 0.9608],
          ...,
          [0.6039, 0.6078, 0.6196,  ..., 0.0118, 0.0118, 0.0118],
          [0.6039, 0.6039, 0.6078,  ..., 0.0078, 0.0078, 0.0078],
          [0.5961, 0.5961, 0.6000,  ..., 0.0078, 0.0078, 0.0078]],
 
         [[0.6471, 0.6510, 0.6588,  ..., 0.8039, 0.7922, 0.7843],
          [0.6471, 0.6510, 0.6588,  ..., 0.8078, 0.7961, 0.7882],
          [0.6431, 0.6471, 0.6588,  ..., 0.8118, 0.8000, 0.7961],
          ...,
          [0.4824, 0.4863, 0.4902,  ..., 0.0118, 0.0118, 0.0118],
          [0.4824, 0.4824, 0.4863,  ..., 0.0078, 0.0078, 0.0078],
          [0.4745, 0.4745, 0.4784,  ..., 0.0078, 0.0078, 0.0078]],
 
         [[0.3412, 0.3451, 0.3569,  ..., 0.4784, 0.4706, 0.4627],
          [0.3412, 0.3451, 0.3569,  ..., 0.4824, 0.4745, 0.4667],
          [0.3412, 0.3451, 0.3569,  ...,

In [61]:
train_set, test_set = torch.utils.data.random_split(dataset, [5,5])

In [62]:
train_loader = DataLoader(dataset = train_set, batch_size = 32,shuffle = True)
test_loader  = DataLoader(dataset = test_set, batch_size = 32, shuffle = True)

## Xây dựng model

In [63]:
model = torchvision.models.googlenet(weights = "DEFAULT")

# freeze all layers, change final linear layer with num_classes
for param in model.parameters():
    param.requires_grad = False

# final layer is not frozen
model.fc = nn.Linear(in_features=1024, out_features = num_class)
model.to(device)

    

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

## Train mô hình

In [64]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [65]:
# Train Network

for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 0.7585830092430115
Cost at epoch 1 is 0.5397429466247559
Cost at epoch 2 is 0.45956239104270935
Cost at epoch 3 is 0.4835180640220642
Cost at epoch 4 is 0.3893641233444214
Cost at epoch 5 is 0.328569233417511
Cost at epoch 6 is 0.2874297499656677
Cost at epoch 7 is 0.34677720069885254
Cost at epoch 8 is 0.2942526936531067
Cost at epoch 9 is 0.2609840929508209


In [66]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set
Got 4 / 5 with accuracy 80.00
Checking accuracy on Test Set
Got 4 / 5 with accuracy 80.00
